In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from tqdm.notebook import trange

from functions import*
from Mloop_and_simcim import*

datatype = torch.float32
device = 'cuda'

In [ ]:
# define number of cities
N_cities = 40
# random generation of city coordinates and their lengths
cities,lengths = generate_cities(N_cities)


B = 0.1
A = 1.5*B*lengths.max()
J,h = get_Jh(lengths, A, B)
J = J.to(device)
h = h.to(device)

In [ ]:
# Сhoose controller type among following options
# ['gaussian_process', 'neural_net', 'differential_evolution', 'nelder_mead', 'random']
controller_type = 'gaussian_process'

#Initilalize simcim interface
simcim = Simcim(J, h, device, datatype)

#params_opt is a dictionary of optimal parameters found by m-loop
params_opt = main(controller_type,simcim)

# run simcim using this dictionary
c_current, c_evol = simcim.evolve(params_opt)

# plotting ammplitudes from a random run
for i in range(c_evol.shape[0]):
    plt.plot(c_evol[i].cpu().numpy())
plt.show()

# Calculating energies of Ising Hamiltonian and plotting a histogram
s_cur = torch.sign(c_current)
E = energy(J,h,s_cur)
plt.hist(E.cpu().numpy())
plt.show()

# choosing a solution with the least energy
# extracting route from that solution
s_min = s_cur[:,torch.argmin(E)]
order = get_order_simcim(s_min,N_cities)

#plotting optimal route
fig = plot_cities(cities,lengths,order.cpu().numpy())